**Credit: [Here](https://www.kaggle.com/nadintamer/titanic-survival-predictions-beginner)**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_pd = pd.read_csv('../input/train.csv')
gender_sub = pd.read_csv('../input/gender_submission.csv')
test_pd = pd.read_csv('../input/test.csv')

In [ ]:
train_pd.describe(include='all')

In [ ]:
sns.barplot(x="Sex", y="Survived", data=train_pd)

print("Percentage of females who survived:", train_pd['Survived'][train_pd['Sex'] == 'female'].value_counts(normalize=True)[1]*100)
print("Percentage of males who survived:", train_pd['Survived'][train_pd['Sex'] == 'male'].value_counts(normalize=True)[1]*100)

In [ ]:
sns.barplot(x='Pclass',y='Survived',data=train_pd)
print('Survival rate of 1st class', train_pd['Survived'][train_pd['Pclass'] == 1].value_counts(normalize=True)[1]*100)
print('Survival rate of 2nd class', train_pd['Survived'][train_pd['Pclass'] == 2].value_counts(normalize=True)[1]*100)
print('Survival rate of 3rd class', train_pd['Survived'][train_pd['Pclass'] == 3].value_counts(normalize=True)[1]*100)

In [ ]:
sns.barplot(x='SibSp', y='Survived',data=train_pd)
print('Percent of SibSp = 0 who survived:', train_pd['Survived'][train_pd['SibSp'] == 0].value_counts(normalize=True)[1]*100)
print('Percent of SibSp = 1 who survived:', train_pd['Survived'][train_pd['SibSp'] == 1].value_counts(normalize=True)[1]*100)
print('Percent of SibSp = 2 who survived:', train_pd['Survived'][train_pd['SibSp'] == 2].value_counts(normalize=True)[1]*100)
print('Percent of SibSp = 3 who survived:', train_pd['Survived'][train_pd['SibSp'] == 3].value_counts(normalize=True)[1]*100)
print('Percent of SibSp = 4 who survived:', train_pd['Survived'][train_pd['SibSp'] == 4].value_counts(normalize=True)[1]*100)

In [ ]:
sns.barplot(x='Parch',y='Survived',data=train_pd)
for i in range(0,7):
    try:
        print('Percentage of Parch =', i, 'who survived:', train_pd['Survived'][train_pd['Parch'] == i].value_counts(normalize=True)[1]*100)
    except:
        pass

In [ ]:
train_pd['Age'] = train_pd['Age'].fillna(-0.5)
test_pd['Age'] = test_pd['Age'].fillna(-0.5)
bins = [-1, 0, 5, 12, 18, 24, 35, 60, np.inf]
labels = ['Unknown', '0-5', '6-12', '13-18', '19-24','25-35','35-60', '60+']
train_pd['Age_Group'] = pd.cut(train_pd['Age'], bins, labels=labels)
test_pd['Age_Group'] = pd.cut(train_pd['Age'], bins, labels=labels)
sns.barplot(x='Age_Group',y='Survived',data=train_pd)
for i in labels:
    print('Percent of Age', i, 'who survived:', train_pd['Survived'][train_pd['Age_Group'] == i].value_counts(normalize=True)[1]*100)

In [ ]:
train_pd['CabinBool'] = train_pd['Cabin'].notnull().astype('int')
test_pd['CabinBool'] = test_pd['Cabin'].notnull().astype('int')
sns.barplot(x='CabinBool',y='Survived',data=train_pd)
print("Percentage of cabinbool = 1 who survived:", train_pd['Survived'][train_pd['CabinBool'] == 1].value_counts(normalize=True)[1]*100)
print("Percentage of cabinbool = 0 who survived:", train_pd['Survived'][train_pd['CabinBool'] == 0].value_counts(normalize=True)[1]*100)

In [ ]:
train_pd = train_pd.drop(['Cabin'], axis = 1)
test_pd = test_pd.drop(['Cabin'], axis = 1)
train_pd = train_pd.drop(['Ticket'], axis = 1)
test_pd = test_pd.drop(['Ticket'], axis = 1)

In [ ]:
train_pd['Embarked']= train_pd['Embarked'].fillna('S')
test_pd['Embarked'] = test_pd['Embarked'].fillna('S')

In [ ]:
combine = [train_pd,test_pd]
for i in combine:
    i['Title'] = i['Name'].str.extract(' ([A-Za-z]+)\.',expand=False)
    i['Title'] = i['Title'].replace('Mlle', 'Miss')
    i['Title'] = i['Title'].replace('Ms', 'Miss')
    i['Title'] = i['Title'].replace('Mme', 'Mrs')

In [ ]:
titles = train_pd[train_pd['Age_Group'] == 'Unknown']['Title'].unique()
title_mapping = {}
for i in titles:
    title_mapping[i] = train_pd[(train_pd['Title'] == i) & (train_pd['Age_Group'] != 'Unknown')]['Age_Group'].mode()[0]
title_mapping

In [ ]:
for x in range(len(train_pd["Age_Group"])):
    if train_pd["Age_Group"][x] == "Unknown":
        train_pd["Age_Group"][x] = title_mapping[train_pd["Title"][x]]
for x in range(len(test_pd["Age_Group"])):
    if test_pd["Age_Group"][x] == "Unknown":
        test_pd["Age_Group"][x] = title_mapping[test_pd["Title"][x]]
test_pd['Age_Group'].value_counts()

In [ ]:
train_pd=train_pd.drop('Name',axis=1)
train_pd=train_pd.drop('Title',axis=1)
train_pd['Sex']=train_pd['Sex'].replace('male',1)
train_pd['Sex']=train_pd['Sex'].replace('female',2)
for j in range(0,len(train_pd['Age_Group'].unique())):
    train_pd['Age_Group']= train_pd['Age_Group'].replace(train_pd['Age_Group'].unique()[j],j+1)
train_pd=train_pd.drop('Age',axis=1)
for j in range(0,len(train_pd['Embarked'].unique())):
    train_pd['Embarked']= train_pd['Embarked'].replace(train_pd['Embarked'].unique()[j],j+1)

In [ ]:
test_pd=test_pd.drop('Name',axis=1)
test_pd=test_pd.drop('Title',axis=1)
test_pd['Sex']=test_pd['Sex'].replace('male',1)
test_pd['Sex']=test_pd['Sex'].replace('female',2)
for j in range(0,len(test_pd['Age_Group'].unique())):
    test_pd['Age_Group']= test_pd['Age_Group'].replace(test_pd['Age_Group'].unique()[j],j+1)
test_pd=test_pd.drop('Age',axis=1)
for j in range(0,len(test_pd['Embarked'].unique())):
    test_pd['Embarked']= test_pd['Embarked'].replace(test_pd['Embarked'].unique()[j],j+1)

In [ ]:
test_pd[test_pd['Fare'].isnull()]['Pclass'] == 3
avgFare = train_pd[train_pd['Pclass'] == 3]['Fare'].mean()
test_pd['Fare'] = test_pd['Fare'].fillna(avgFare)
test_pd.describe()

In [ ]:
from sklearn.model_selection import train_test_split
predictors = train_pd.drop(['Survived', 'PassengerId'], axis = 1)
target = train_pd['Survived']
xtrain, xtest, ytrain, ytest = train_test_split(predictors, target, test_size = .22, random_state = 42)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

gaussian = GaussianNB()
gaussian.fit(xtrain, ytrain)
ypred = gaussian.predict(xtest)
gaussian_acc = accuracy_score(ypred, ytest)
gaussian_acc

In [ ]:
from sklearn.linear_model import LogisticRegression
logrec = LogisticRegression()
logrec.fit(xtrain, ytrain)
ypred = logrec.predict(xtest)
logrec_acc = accuracy_score(ytest,ypred)
logrec_acc

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(xtrain,ytrain)
ypred = svc.predict(xtest)
svc_acc = accuracy_score(ytest, ypred)
svc_acc

In [ ]:
from sklearn.linear_model import Perceptron
perceptron = Perceptron()
perceptron.fit(xtrain,ytrain)
ypred = perceptron.predict(xtest)
perceptron_acc = accuracy_score(ytest,ypred)
perceptron_acc

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(xtrain,ytrain)
ypred = dtc.predict(xtest)
dtc_acc = accuracy_score(ytest,ypred)
dtc_acc

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(xtrain,ytrain)
ypred = rfc.predict(xtest)
rfc_acc = accuracy_score(ytest,ypred)
rfc_acc

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(xtrain,ytrain)
ypred = knn.predict(xtest)
knn_acc = accuracy_score(ytest, ypred)
knn_acc

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(random_state = 5)
sgd.fit(xtrain,ytrain)
ypred = sgd.predict(xtest)
sgd_acc = accuracy_score(ytest,ypred)
sgd_acc

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbk = GradientBoostingClassifier()
gbk.fit(xtrain,ytrain)
ypred = gbk.predict(xtest)
gbk_acc = accuracy_score(ytest, ypred)
gbk_acc

In [ ]:
models = pd.DataFrame({
    'Model' : ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Decision Tree', 'Stochastic Gradient Descent', 'Gradient Boosting Classifier'],
    'Accuracy': [svc_acc, knn_acc, logrec_acc, rfc_acc, gaussian_acc, perceptron_acc, dtc_acc, sgd_acc, gbk_acc]
})
models.sort_values(by='Accuracy', ascending=False)

In [ ]:
gbk = GradientBoostingClassifier()
gbk.fit(train_pd.drop('Survived',axis=1), train_pd['Survived'])
ids = test_pd['PassengerId']
predictions = gbk.predict(test_pd)
output = pd.DataFrame({'PassengerId' : ids, 'Survived' : predictions})
output.to_csv('submission.csv', index=False)